### Task 1 - Data Scraping and Collection (Extract & Load)

In [15]:
# %pip install telethon
# %pip install psycopg2
# %pip install pandas
# %pip install matplotlib
# %pip install numpy
# %pip install psycopg2
# %pip install dotenv

In [21]:
import os
import sys
import asyncio
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from telethon.sync import TelegramClient
from dotenv import load_dotenv
import numpy as np
import json
from pathlib import Path
from datetime import datetime
# Load .env file from the project root
load_dotenv(dotenv_path="../.env")  # adjust the path if needed
import logging
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("..") 
print(os.getcwd())

d:\KAIM 5,6,7\Week-7\Technical\Shipping-Data-from-Raw-Telegram-Data


In [22]:
# Access the variables
api_id = os.getenv("tg_api_id")
api_hash = os.getenv("tg_api_hash")
session = os.getenv("session_name")

channels = {
    "lobelia4cosmetics": "https://t.me/lobelia4cosmetics",
    "tikvahpharma": "https://t.me/tikvahpharma"
    # Add more from et.tgstat.com/medicine
}

raw_dir = Path("Data/raw/telegram_messages")

logging.basicConfig(filename="scrape.log", level=logging.INFO)

async def scrape_channel(channel_name, max_msgs=1000):
    client = TelegramClient("async_session", api_id, api_hash)
    try:
        await client.start()
        messages = []
        async for msg in client.iter_messages(channel_name, limit=max_msgs):
            messages.append(msg.to_dict())
        logging.info(f"[{channel_name}] Scraped {len(messages)} messages.")

        # Save JSON
        today = datetime.now().strftime("%Y-%m-%d")
        out_dir = raw_dir / today / channel_name
        out_dir.mkdir(parents=True, exist_ok=True)
        with open(out_dir / "messages.json", "w", encoding="utf-8") as f:
            json.dump(messages, f, ensure_ascii=False, indent=2)

    except Exception as e:
        logging.error(f"Error scraping {channel_name}: {e}")

In [23]:
await run()